<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#工具包-&amp;-数据导入" data-toc-modified-id="工具包-&amp;-数据导入-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>工具包 &amp; 数据导入</a></span><ul class="toc-item"><li><span><a href="#开源工具包导入" data-toc-modified-id="开源工具包导入-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>开源工具包导入</a></span></li><li><span><a href="#自定义工具包导入" data-toc-modified-id="自定义工具包导入-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>自定义工具包导入</a></span></li><li><span><a href="#数据导入" data-toc-modified-id="数据导入-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>数据导入</a></span><ul class="toc-item"><li><span><a href="#原始数据读取" data-toc-modified-id="原始数据读取-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>原始数据读取</a></span></li><li><span><a href="#降低数据内存" data-toc-modified-id="降低数据内存-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>降低数据内存</a></span></li></ul></li></ul></li><li><span><a href="#特征工程" data-toc-modified-id="特征工程-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>特征工程</a></span><ul class="toc-item"><li><span><a href="#基于AvSigVersion的特征" data-toc-modified-id="基于AvSigVersion的特征-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>基于AvSigVersion的特征</a></span></li><li><span><a href="#数据合并" data-toc-modified-id="数据合并-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>数据合并</a></span></li></ul></li><li><span><a href="#模型训练&amp;测试" data-toc-modified-id="模型训练&amp;测试-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>模型训练&amp;测试</a></span><ul class="toc-item"><li><span><a href="#全量LGB" data-toc-modified-id="全量LGB-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>全量LGB</a></span></li><li><span><a href="#模型训练" data-toc-modified-id="模型训练-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>模型训练</a></span></li><li><span><a href="#模型测试" data-toc-modified-id="模型测试-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>模型测试</a></span></li><li><span><a href="#实验结果(LB:0.69985,-PB：0.61388,-57/2426)" data-toc-modified-id="实验结果(LB:0.69985,-PB：0.61388,-57/2426)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>实验结果(LB:0.69985, PB：0.61388, 57/2426)</a></span></li></ul></li><li><span><a href="#小结" data-toc-modified-id="小结-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>小结</a></span></li></ul></div>

# 工具包 & 数据导入
## 开源工具包导入

In [1]:
## 数据工具包
import pandas as pd
import numpy as np
np.random.seed(42)

from tqdm import tqdm 
from tqdm import tqdm_notebook
## 字符串处理工具包

import string
import re
import gensim
from collections import Counter
import pickle
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.preprocessing import text, sequence 

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
import lightgbm as lgb
from functools import partial
import joblib

import os 
import gc
from scipy.sparse import vstack  
import time 
import seaborn as sns
%matplotlib inline

/opt/conda/envs/zjpy36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 自定义工具包导入
- 该工具包主要用于降低数据的内存消耗,但是在使用时需要注意精度溢出的问题


In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm 
class _Data_Preprocess:
    def __init__(self):
        self.int8_max =  np.iinfo(np.int8).max
        self.int8_min =  np.iinfo(np.int8).min
        
        self.int16_max =  np.iinfo(np.int16).max
        self.int16_min =  np.iinfo(np.int16).min
      
        self.int32_max =  np.iinfo(np.int32).max
        self.int32_min =  np.iinfo(np.int32).min
      
        self.int64_max =  np.iinfo(np.int64).max
        self.int64_min =  np.iinfo(np.int64).min
        
        self.float16_max =  np.finfo(np.float16).max
        self.float16_min =  np.finfo(np.float16).min
        
        self.float32_max =  np.finfo(np.float32).max
        self.float32_min =  np.finfo(np.float32).min
      
        self.float64_max =  np.finfo(np.float64).max
        self.float64_min =  np.finfo(np.float64).min
         
    
    '''
    function: _get_type(self,min_val, max_val, types)
    
       get the correct types that our columns can trans to
    
    '''
    def _get_type(self,min_val, max_val, types):
        if types == 'int':
            if max_val <= self.int8_max and min_val >= self.int8_min:
                return np.int8
            elif max_val <= self.int16_max  <= max_val and min_val >= self.int16_min:
                return np.int16
            elif max_val <= self.int32_max and min_val >= self.int32_min:
                return np.int32
            return None
            
        elif types == 'float':
            if max_val <=  self.float16_max and min_val >= self.float16_min :
                return np.float16
            if max_val <=  self.float32_max and min_val >= self.float32_min :
                return np.float32
            if max_val <=  self.float64_max and min_val >= self.float64_min:
                return np.float64 
            return None
    '''
    
    function: _memory_process(self,df) 
       column data types trans, to save more memory
    '''
    def _memory_process(self,df):
        init_memory = df.memory_usage().sum() / 1024**2 / 1024
        print('Original data occupies {} GB memory.'.format(init_memory))
        df_cols = df.columns
        
        for col in tqdm(df_cols):
            try:
                if 'float' in str(df[col].dtypes): 
                    max_val = df[col].max()
                    min_val = df[col].min()
                    trans_types = self._get_type(min_val,max_val,'float')
                    if trans_types is not None: 
                        df[col] = df[col].astype(trans_types)
                elif 'int' in str(df[col].dtypes):  
                    max_val = df[col].max()
                    min_val = df[col].min()
                    trans_types = self._get_type(min_val,max_val,'int')
                    if trans_types is not None:  
                        df[col] = df[col].astype(trans_types) 
            except:
                print(' Can not do any process for column, {}.'.format(col))
        afterprocess_memory = df.memory_usage().sum() / 1024**2 / 1024
        print('After processing, the data occupies {} GB memory.'.format(afterprocess_memory))
        return df

 ## 数据导入
 ### 原始数据读取

In [3]:
path = '/data/Data_JieZhang/MicroMalware/'

In [4]:
%%time
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')

CPU times: user 3min 8s, sys: 14.5 s, total: 3min 23s
Wall time: 3min 31s


In [5]:
test['HasDetections'] = -1

In [6]:
train_test = pd.concat([train,test],ignore_index = True)

### 降低数据内存
- 因为数据内存较大,如果机器内存较小,可以考虑先对数据的内存进行处理,当然还有一种更加方便的策略,就是在读取数据的时候就指定每列特征是int还是float型,只是那样得比较细心,各位可以按照自己的喜好进行处理。

In [7]:
data_preprocess = _Data_Preprocess()

In [8]:
%%time
train_test = data_preprocess._memory_process(train_test) 

  0%|          | 0/83 [00:00<?, ?it/s]

Original data occupies 10.373466446995735 GB memory.


100%|██████████| 83/83 [00:14<00:00,  3.89it/s]

After processing, the data occupies 5.749150112271309 GB memory.
CPU times: user 13.7 s, sys: 736 ms, total: 14.4 s
Wall time: 14.3 s


# 特征工程
## 基于AvSigVersion的特征

- 我们对每个AvSigVersion做强特,该特征可以反应在每个版本上不同的其他信息所占的比例, 用其他类在每个版本上的分布更好的来反映版本的情况。


In [9]:
cols2 = ['Census_OSUILocaleIdentifier','AVProductsInstalled','Census_FirmwareVersionIdentifier','Wdft_IsGamer',
                         'Census_ThresholdOptIn','RtpStateBitfield','Census_IsSecureBootEnabled','AVProductsEnabled',
                         'HasTpm','IsProtected','Census_PrimaryDiskTypeName','PuaMode','DefaultBrowsersIdentifier','IsSxsPassiveMode',
                         'OrganizationIdentifier', 'Census_IsAlwaysOnAlwaysConnectedCapable','ProductName','GeoNameIdentifier','Census_IsVirtualDevice',
                         'Census_PowerPlatformRoleName', 'Census_IsTouchEnabled','Census_OSSkuName','OsPlatformSubRelease',
                         'Census_FlightRing','Census_OSEdition','Census_IsPortableOperatingSystem','Firewall',
                         'OsBuildLab','Census_DeviceFamily','Census_IsPenCapable','SMode', 'Platform', 'Census_IsFlightingInternal',
                         'Census_OEMNameIdentifier','Census_InternalBatteryType','OsBuild','Census_HasOpticalDiskDrive','Census_IsWIMBootEnabled',
                         'Census_OSBuildRevision','CityIdentifier','IeVerIdentifier','Census_ProcessorClass', 'OsSuite','Census_IsFlightsDisabled',
                         'Census_ChassisTypeName','LocaleEnglishNameIdentifier','Census_OSArchitecture',
                         'CountryIdentifier','Census_OSInstallLanguageIdentifier','Census_OSInstallTypeName',
                         'Census_OSBuildNumber','AutoSampleOptIn', 'OsVer','SkuEdition','UacLuaenable','Census_OEMModelIdentifier','Census_OSBranch',
                         'Processor','Census_ProcessorModelIdentifier', 'Census_ActivationChannel','IsBeta', 'Census_MDC2FormFactor', 'Census_OSWUAutoUpdateOptionsName',
                         'AVProductStatesIdentifier','Census_GenuineStateName','Census_FirmwareManufacturerIdentifier',
                         'Wdft_RegionIdentifier','Census_ProcessorManufacturerIdentifier', 
                         'EngineVersion','AppVersion','Census_OSVersion',]

col1 = 'AvSigVersion'

In [17]:
AvSig_based_fea = pd.DataFrame()
AvSig_based_fea['MachineIdentifier'] = train_test['MachineIdentifier'].values
train_test[col1 + 'cnt'] = train_test[col1].fillna(-999).map(train_test[col1].fillna(-999).value_counts())

for col2 in tqdm_notebook(cols2):
    col = [col1, col2]
    train_test['tmp'] = train_test[col1].fillna('NAN').astype(str) + '_' + train_test[col2].fillna('NAN').astype(str)
    AvSig_based_fea[col1 + '_' + col2]  = train_test['tmp'].map(train_test['tmp'].value_counts()) / train_test[col1 + 'cnt']
del train_test['tmp']

HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

- 降低数据的内存使用

In [18]:
%%time
AvSig_based_fea = data_preprocess._memory_process(AvSig_based_fea) 

  0%|          | 0/72 [00:00<?, ?it/s]

Original data occupies 8.99866969883442 GB memory.


100%|██████████| 72/72 [00:22<00:00,  3.14it/s]

After processing, the data occupies 2.343403622508049 GB memory.
CPU times: user 18.5 s, sys: 4.02 s, total: 22.5 s
Wall time: 22.4 s


 ## 数据合并

In [19]:
import joblib
train_set = joblib.load('train_set.jl.z')
test_set  = joblib.load('test_set.jl.z') 

In [20]:
%%time
train_set = train_set.merge(AvSig_based_fea, on = 'MachineIdentifier', how='left')
test_set  = test_set.merge(AvSig_based_fea,  on = 'MachineIdentifier', how='left')

CPU times: user 1min 20s, sys: 21.1 s, total: 1min 41s
Wall time: 1min 41s


# 模型训练&测试

- 因为此次数据的量较大,采用5折的LGB训练虽然可以取得不错的成绩,但是迭代的过程较慢,而且数据量非常的大,不是一个很好的选择,当然如果大家希望在比赛过程中获得更好的成绩,建议大家采用N-fold的训练测试的方法。

## 全量LGB

In [21]:
def _single_lgbmodel(df, df_label):
    importances = pd.DataFrame()
    lgb_params = {'objective':'binary', "boosting": "gbdt", 'learning_rate': 0.03, 'max_depth': -1, 
     "feature_fraction": 0.8, "bagging_freq": 1, "bagging_fraction": 0.8 , "bagging_seed": 11, 'n_estimators': 2500,
     "metric": 'auc', "lambda_l1": 0.1, 'num_leaves': 256, 'min_data_in_leaf': 50, "verbose": 1, "num_threads": 50}
    
    model = lgb.LGBMClassifier(**lgb_params)
    model.fit(df, df_label, eval_set=[(df, df_label)], verbose = 250, eval_metric ='auc')     

    imp_df = pd.DataFrame()
    imp_df['feature'] = df.columns
    imp_df['gain']    = model.feature_importances_
        
    gc.collect()
    return model,imp_df 

## 模型训练

In [23]:
train_cols = [col for col in train_set.columns if col not in ['MachineIdentifier','HasDetections','isval']]
train_label = 'HasDetections'

In [24]:
%%time
model, model_imp = _single_lgbmodel(train_set[train_cols], train_set[train_label].values)

[250]	training's auc: 0.734849
[500]	training's auc: 0.744285
[750]	training's auc: 0.74914
[1000]	training's auc: 0.752887
[1250]	training's auc: 0.756324
[1500]	training's auc: 0.759525
[1750]	training's auc: 0.762635
[2000]	training's auc: 0.765652
[2250]	training's auc: 0.768543
[2500]	training's auc: 0.771496
CPU times: user 1d 22h 52min 55s, sys: 1d 4h 26min 8s, total: 3d 3h 19min 4s
Wall time: 2h 21min 9s


In [27]:
model_imp.sort_values('gain',ascending = False)

,feature,gain
4,AVProductStatesIdentifier,12071
90,CountryIdentifier_cnt,10708
8,CountryIdentifier,9838
185,AvSigVersion_Census_OSInstallTypeName,9530
139,AvSigVersion_Wdft_IsGamer,9278
111,Census_SystemVolumeTotalCapacity_cnt,8500
205,AvSigVersion_AppVersion,8352
9,CityIdentifier,8263
195,AvSigVersion_Census_ActivationChannel,8020
91,CityIdentifier_cnt,8004


## 模型测试

In [25]:
pred = model.predict_proba(test_set[train_cols])[:,1]
test_set['HasDetections'] = pred
test_set['HasDetections'] = test_set['HasDetections']
test_set[['MachineIdentifier','HasDetections']].to_csv('baseline_labelencode_frequency_avsgencode.csv',index = None)

## 实验结果(LB:0.69985, PB：0.61388, 57/2426)
![](./pic/baseline_avsgencode_score.png)

# 小结

在本篇文章中,我们在上一节的基础之上加入了基于AvSig的强特,该特征为何有效,建议大家看一下discussion区的分析,理解该特征的有效性对于之后很多比赛都会带来极大的帮助, <font color=blue>https://www.kaggle.com/c/microsoft-malware-prediction/discussion/84287 </font>, 该部分的理解就作为大家的课后作业吧,

- <font color=red> **大家将上面的代码改成5-fold的代码之后就可以到0.701左右的分数了。 **</font>

至于单模上0.705,大家把自己的特征加上我上面的特征进行训练应该就可以达到LB金牌区的分数了。在后面的文章中，我会再写一篇文章介绍如何在PB榜取得前排的文章。
